In [ ]:
from threading import Thread
import warnings
import os
from xmlrpc.server import SimpleXMLRPCServer

import numpy as np
import specpy as sp
import h5py as h5

import pipeline2 as p2
from pipeline2.tiling import centered_tiles

class ImspectorAcquisition():

    def __init__(self, save_dir, file_prefix) -> None:
        self.imspector = sp.get_application()

        # TODO: catch no open measurement
        self.parameters = self.imspector.active_measurement().active_configuration().parameters('')

        self.position = [self.imspector.active_measurement().active_configuration().parameters('ExpControl/scan/range/coarse_' + c + '/g_off') for c in 'xy']
        self.field_of_view = [self.imspector.active_measurement().active_configuration().parameters('ExpControl/scan/range/' + c + '/len') for c in 'xy']
        self.overlap = 0.1

        self.save_path = save_dir
        self.file_prefix = file_prefix

        self.n_tiles = 1
        self.time_points = None

        self.pipeline_thread = None

    def start_pipeline_thread(self):

        if self.pipeline_thread is not None:
            raise ValueError('a measurement pipeline is running currently, finish that first.')

        self.pipeline_thread = Thread(target=self.run_pipeline)
        self.pipeline_thread.start()

    @staticmethod
    def get_smallest_numeric_suffix_for_files(base_h5_path):
        suffix = 0
        while os.path.exists(base_h5_path.replace('.h5', f'{suffix}.h5')):
            suffix += 1
        return suffix

    def run_pipeline(self):

        # make output dir if it does not exist already
        if not os.path.exists(self.save_path):
            os.makedirs(self.save_path)

        levels = ()
        # add timepoint level if we want timepoints
        if self.time_points is not None:
            levels += ('timepoint', )
        # call second level tile if we have tiles, else image
        levels += (('tile', ) if np.max(self.n_tiles) > 1 else ('image', ))

        pll = p2.PipelineLevels(*levels)

        # make name handler, add suffix to prevent overwriting exisiting files
        numeric_suffix = ImspectorAcquisition.get_smallest_numeric_suffix_for_files(os.path.join(self.save_path, self.file_prefix + '.h5'))
        name_handler = p2.DefaultNameHandler(self.save_path, pll, prefix=f'{self.file_prefix}{numeric_suffix}')

        # init pipeline
        pl = (p2.AcquisitionPipeline('test-pipeline')
                .withImspectorConnection(p2.imspector.ImspectorConnection(self.imspector).withVerbose())
                .withPipelineLevels(pll)
                .withNameHandler(name_handler)
            )

        tile_positions = centered_tiles(self.position, self.field_of_view, self.n_tiles, self.overlap)
        tile_generator = p2.taskgeneration.StagePositionListGenerator(tile_positions)
        atg_images = p2.taskgeneration.AcquisitionTaskGenerator(pll.levels[-1],
            p2.taskgeneration.DefaultLocationRemover(p2.taskgeneration.JSONFileConfigLoader([self.parameters])),
            p2.taskgeneration.DefaultStageOffsetsSettingsGenerator(tile_generator, True, tile_generator.get_all_locations)
        )

        if self.time_points is not None:
            timeseries_callback = p2.taskgeneration.taskgeneration.TimeSeriesCallback(pll.timepoint)
            timeseries_callback.time_points = self.time_points
            pl.withCallbackAtLevel(timeseries_callback, pll.timepoint)

            # also add tile callback
            pl.withCallbackAtLevel(atg_images, pll.timepoint)

        h5file_name = name_handler.get_path([]).replace('.msr', '.h5')

        # open resulting h5 file in context manager
        with h5.File(h5file_name, mode='a') as h5fd:
            data_store = p2.HDF5DataStore(h5fd, pll)
            pl.withDataStorage(data_store)

            if self.time_points is not None:
                timeseries_callback.initialize_time_series(pl)
            else:
                atg_images(pl)

            pl.run()


    def set_parameters_and_start_acquisition(self, n_tiles, time_points):

        self.n_tiles = n_tiles
        self.time_points = time_points

        self.start_pipeline_thread()

    def finish_acquisition(self):
        if self.pipeline_thread is None:
            return
        self.pipeline_thread.join()
        self.pipeline_thread = None

    def run_server(self, host, port):
        with SimpleXMLRPCServer((host, port), allow_none=True) as server:
            server.register_function(self.finish_acquisition, 'finish_acquisition')
            server.register_function(self.set_parameters_and_start_acquisition, 'run_acquisition')
            server.serve_forever()



## Tests

In [ ]:
# setup acquisition

save_path = 'D:/AUTOMATION/TEST/'
file_prefix = 'tiletest'
acq = ImspectorAcquisition(save_path, file_prefix)

### Run blocking

In [ ]:
acq.time_points = [0, 10, 20]
acq.n_tiles = (2, 2)
acq.overlap = 0.25
acq.run_pipeline()

### Run in seperate thread

In [ ]:
acq.time_points = [0, 10, 20]
acq.n_tiles = (1, 2)
acq.start_pipeline_thread()

In [ ]:
acq.finish_acquisition()

### Run as XMLRPC server

In [ ]:
acq.run_server('localhost', 11005)